### Build concat TS and connectome with chunks

In [11]:
import os
import pickle
import numpy as np
from nilearn import connectome
from sklearn.covariance import LedoitWolf
from sklearn.preprocessing import StandardScaler

# === Configuration ===
dimension_of_atlas = 64
chunked_dir = "timeseries"
min_timepoints = 10
connectome_method = "tangent"
discard_diagonal = False

# === Load chunked data ===
all_chunks = []
all_chunk_ids = []

for filename in os.listdir(chunked_dir):
    if filename.endswith(".pkl"):
        subject_id = filename.replace(".pkl", "")
        with open(os.path.join(chunked_dir, filename), "rb") as f:
            chunks = pickle.load(f)

for filename in os.listdir(chunked_dir):
    if filename.endswith(".pkl"):
        subject_id = filename.replace(".pkl", "")
        with open(os.path.join(chunked_dir, filename), "rb") as f:
            chunks = pickle.load(f)
        for idx, chunk in enumerate(chunks):
            if chunk.shape[0] < min_timepoints:
                continue
            all_chunks.append(chunk)
            all_chunk_ids.append(f"{subject_id}_chunk{idx+1}")

print(f"✅ Loaded {len(all_chunks)} usable chunks from {len(all_chunk_ids)} total chunk IDs.")

# === Save all time series (with IDs) ===
ts_output_path = f"timeseries/ts_{dimension_of_atlas}.pkl"
with open(ts_output_path, "wb") as f:
    pickle.dump((all_chunks, all_chunk_ids), f)
print(f"📦 Saved concatenated time series to {ts_output_path}")

# === Compute connectomes ===
connectome_measure = connectome.ConnectivityMeasure(
    cov_estimator=LedoitWolf(assume_centered=True),
    kind=connectome_method,
    discard_diagonal=discard_diagonal,
    vectorize=True
)

X = connectome_measure.fit_transform(all_chunks)
scaler = StandardScaler()
X = scaler.fit_transform(X)
print(f"✅ Computed connectomes with shape: {X.shape}")

# === Save connectomes and IDs ===
connectome_path = f"connectome/connectomes_{dimension_of_atlas}.npy"
ids_path = f"connectome/connectome_ids_{dimension_of_atlas}.pkl"

np.save(connectome_path, X)
with open(ids_path, "wb") as f:
    pickle.dump(all_chunk_ids, f)

print(f"📦 Saved vectorized connectomes to {connectome_path}")
print(f"📦 Saved corresponding IDs to {ids_path}")


✅ Loaded 140 usable chunks from 140 total chunk IDs.
📦 Saved concatenated time series to timeseries/ts_64.pkl
✅ Computed connectomes with shape: (140, 2080)
📦 Saved vectorized connectomes to connectome/connectomes_64.npy
📦 Saved corresponding IDs to connectome/connectome_ids_64.pkl


In [10]:
import pickle
import numpy as np

# === Path to your .pkl file ===
pkl_path = "timeseries/HCA6058970.pkl"

# === Load the file ===
with open(pkl_path, "rb") as f:
    data = pickle.load(f)

# === Display summary ===
print(f"🔍 Loaded {len(data)} chunks")

for idx, chunk in enumerate(data):
    chunk = np.asarray(chunk)
    print(f"\n🧩 Chunk {idx+1}: shape {chunk.shape}")
    print(chunk)  # or print(chunk[:5]) to see first 5 rows

    if idx == 2:
        break  # Show only first 3 chunks to keep output manageable

🔍 Loaded 10 chunks

🧩 Chunk 1: shape (191, 64)
[[ 1.4435895e-01 -6.3068546e-02  1.3269721e-02 ...  5.8984172e-02
  -8.6618111e-02 -1.2519098e-02]
 [-1.2186605e+00 -2.2169331e-01  1.2257959e+00 ... -2.5013125e+00
  -5.0428283e-01  6.4696658e-01]
 [-1.7477064e+00 -1.0675676e+00  4.3724680e-01 ... -2.9192250e+00
  -8.4240866e-01  6.9413352e-04]
 ...
 [-1.5882313e-01 -2.3469646e-02  9.8152024e-01 ... -2.2017385e-01
   1.0668461e+00  5.0219625e-01]
 [-5.5617493e-01  1.3906076e+00  1.0456649e+00 ...  1.1302136e+00
   7.4152881e-01 -5.8765692e-01]
 [-8.1274837e-01  1.7046607e+00  9.0575117e-01 ...  2.0316153e+00
  -4.3403409e-02 -6.1118990e-01]]

🧩 Chunk 2: shape (191, 64)
[[-0.5154104   0.7496681   0.48824584 ...  1.855288   -0.03866327
   0.63850105]
 [ 0.31425062  0.17197126  0.1687282  ...  0.9502816   0.34955123
   1.4889023 ]
 [ 0.7920628   0.38329828  0.1866765  ...  0.01667785  0.23206249
   1.0020478 ]
 ...
 [ 0.32825625  0.49986407  0.03864482 ...  1.2517389  -0.18494828
   0.184698

### DISPLAY DATA

In [6]:
import pickle
import numpy as np

dimension_of_atlas = 64  # or 256, 64, etc., depending on your setting

# === Load raw time series and IDs ===
with open(f"timeseries/ts_{dimension_of_atlas}.pkl", "rb") as f:
    time_series_chunks, chunk_ids = pickle.load(f)

print("🧠 First 10 chunk IDs:")
print(chunk_ids[:10])

print("\n🕒 First 10 time series shapes:")
for ts in time_series_chunks[:10]:
    print(ts.shape)

# === Load vectorized connectomes ===
connectomes = np.load(f"connectome/{dimension_of_atlas}/connectomes_{dimension_of_atlas}.npy")
print("\n🧬 First 10 connectome vectors (shape per vector):")
print(connectomes[:10])
print("Each connectome vector shape:", connectomes[0].shape)

# === Load connectome IDs ===
with open(f"Results/{dimension_of_atlas}/connectome_ids_{dimension_of_atlas}.pkl", "rb") as f:
    connectome_ids = pickle.load(f)

print("\n🔗 First 10 connectome IDs:")
print(connectome_ids[:10])


🧠 First 10 chunk IDs:
['HCA6089375_chunk1', 'HCA6089375_chunk2', 'HCA6089375_chunk3', 'HCA6089375_chunk4', 'HCA6089375_chunk5', 'HCA6089375_chunk6', 'HCA6089375_chunk7', 'HCA6089375_chunk8', 'HCA6089375_chunk9', 'HCA6089375_chunk10']

🕒 First 10 time series shapes:
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(191, 64)
(193, 64)


FileNotFoundError: [Errno 2] No such file or directory: 'Results/64/connectomes_64.npy'